In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
df_postcodes = pd.read_csv('sunshine_coast.csv')
df_coords = pd. read_csv('Australian_Post_Codes_Lat_Lon.csv')

In [3]:
df_postcodes['Places'] = df_postcodes['Places'].str.upper()

In [4]:
df_merge = df_postcodes.merge(df_coords, left_on = 'Places', right_on = 'suburb', how = 'left')

In [5]:
df_merge.head()

,Places,Postcode,Unnamed: 2,Streets,postcode,suburb,state,dc,type,lat,lon
0,ALEXANDRA HEADLAND,4572,NaN,51,4572.0,ALEXANDRA HEADLAND,QLD,MAROOCHYDORE DC,Delivery Area,-26.670635,153.108029
1,AROONA,4551,NaN,69,4551.0,AROONA,QLD,CALOUNDRA DC,Delivery Area,-26.775377,153.113173
2,BALD KNOB,4552,NaN,14,4552.0,BALD KNOB,QLD,NAMBOUR DC,Delivery Area,-26.775957,152.892993
3,BALMORAL RIDGE,4552,NaN,14,4552.0,BALMORAL RIDGE,QLD,NAMBOUR DC,Delivery Area,-26.740350,152.896247
4,BARINGA,4551,NaN,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_merge.drop(['Unnamed: 2', 'Streets', 'state','type', 'postcode','dc', 'suburb'], axis = 1, inplace = True)
df_merge.head()

,Places,Postcode,lat,lon
0,ALEXANDRA HEADLAND,4572,-26.670635,153.108029
1,AROONA,4551,-26.775377,153.113173
2,BALD KNOB,4552,-26.775957,152.892993
3,BALMORAL RIDGE,4552,-26.740350,152.896247
4,BARINGA,4551,NaN,NaN


In [7]:
df_merge = df_merge.dropna()
df_merge.head()

,Places,Postcode,lat,lon
0,ALEXANDRA HEADLAND,4572,-26.670635,153.108029
1,AROONA,4551,-26.775377,153.113173
2,BALD KNOB,4552,-26.775957,152.892993
3,BALMORAL RIDGE,4552,-26.740350,152.896247
5,BATTERY HILL,4551,-26.777639,153.134820


In [8]:
CLIENT_ID = 'EJHQVEKLIB23DAGXTC5XP5UWN0ICRGEZ4IIWNTFAX5WDKXGQ' # your Foursquare ID
CLIENT_SECRET = 'QG50IGQGHBBU3DVHI2SKYQXWJTHR00CYVZIIJHZN1G5RELF4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EJHQVEKLIB23DAGXTC5XP5UWN0ICRGEZ4IIWNTFAX5WDKXGQ
CLIENT_SECRET:QG50IGQGHBBU3DVHI2SKYQXWJTHR00CYVZIIJHZN1G5RELF4


In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [11]:
Sunshine_venues = getNearbyVenues(names=df_merge['Places'],
                                   latitudes=df_merge['lat'],
                                   longitudes=df_merge['lon']
                                  )

ALEXANDRA HEADLAND
AROONA
BALD KNOB
BALMORAL RIDGE
BATTERY HILL
BEERBURRUM
BEERWAH
BELLI PARK
BELLS CREEK
BIRTINYA
BLACK MOUNTAIN
BLACK MOUNTAIN
BLACK MOUNTAIN
BLI BLI
BOKARINA
BOOROOBIN
BOREEN POINT
BRIDGES
BUDDINA
BUDERIM
BURNSIDE
BURNSIDE
BURNSIDE
BURNSIDE
CALOUNDRA
CALOUNDRA WEST
CAMBROON
CASTAWAYS BEACH
CHEVALLUM
COES CREEK
COMO
COMO
COMO
COMO
CONONDALE
COOCHIN CREEK
COOLABINE
COOLOOLABIN
COOLUM BEACH
COORAN
COOROIBAH
COOROY
COOROY MOUNTAIN
COOTHARABA
CROHAMHURST
CURRAMORE
CURRAMORE
CURRIMUNDI
DIAMOND VALLEY
DICKY BEACH
DIDDILLIBAH
DOONAN
DULONG
EERWAH VALE
ELAMAN CREEK
EUDLO
EUMUNDI
FLAXTON
FOREST GLEN
FOREST GLEN
GHEERULLA
GLASS HOUSE MOUNTAINS
GLENVIEW
GOLDEN BEACH
GOLDEN BEACH
HIGHWORTH
HUNCHY
ILKLEY
IMAGE FLAT
KENILWORTH
KIAMBA
KIDAMAN CREEK
KIELS MOUNTAIN
KINGS BEACH
KIN KIN
KULANGOOR
KULUIN
KUNDA PARK
KUREELPA
LAKE MACDONALD
LANDERS SHOOT
LANDSBOROUGH
LANDSBOROUGH
LITTLE MOUNTAIN
MALENY
MAPLETON
MARCOOLA
MARCUS BEACH
MAROOCHYDORE
MAROOCHY RIVER
MERIDAN PLAINS
MINYAMA
MOFFAT

In [12]:
Sunshine_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALEXANDRA HEADLAND,-26.670635,153.108029,Alexandra Headland,-26.670426,153.107478,Surf Spot
1,ALEXANDRA HEADLAND,-26.670635,153.108029,Alex Surf Club,-26.668784,153.107421,Sports Bar
2,ALEXANDRA HEADLAND,-26.670635,153.108029,Black Bunny Kitchen,-26.670303,153.107370,Café
3,ALEXANDRA HEADLAND,-26.670635,153.108029,Cold Rock Ice Creamery,-26.670065,153.107215,Ice Cream Shop
4,ALEXANDRA HEADLAND,-26.670635,153.108029,Milk & Beans Coffee House,-26.668563,153.106662,Café


In [13]:
Sunshine_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALEXANDRA HEADLAND,7,7,7,7,7,7
AROONA,2,2,2,2,2,2
BALD KNOB,4,4,4,4,4,4
BALMORAL RIDGE,1,1,1,1,1,1
BATTERY HILL,4,4,4,4,4,4
BEERWAH,12,12,12,12,12,12
BIRTINYA,8,8,8,8,8,8
BLACK MOUNTAIN,4,4,4,4,4,4
BLI BLI,5,5,5,5,5,5


In [14]:
# one hot encoding
Sunshine_onehot = pd.get_dummies(Sunshine_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Sunshine_onehot['Neighborhood'] = Sunshine_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Sunshine_onehot.columns[-1]] + list(Sunshine_onehot.columns[:-1])
Sunshine_onehot = Sunshine_onehot[fixed_columns]

Sunshine_onehot.head()

,Neighborhood,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Big Box Store,Boat Launch,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Golf Course,Grocery Store,Gym,Harbor / Marina,Health & Beauty Service,Home Service,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Lake,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Outdoor Supply Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Resort,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Park,Tourist Information Center,Toy / Game Store,Train Station,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Water Park
0,ALEXANDRA HEADLAND,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,ALEXANDRA HEADLAND,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ALEXANDRA HEADLAND,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ALEXANDRA HEADLAND,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ALEXANDRA HEADLAND,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
sunshine_grouped = Sunshine_onehot.groupby('Neighborhood').mean().reset_index()
sunshine_grouped

,Neighborhood,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Big Box Store,Boat Launch,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Golf Course,Grocery Store,Gym,Harbor / Marina,Health & Beauty Service,Home Service,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Lake,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Outdoor Supply Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Resort,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Park,Tourist Information Center,Toy / Game Store,Train Station,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Water Park
0,ALEXANDRA HEADLAND,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.142857,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,AROONA,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,BALD KNOB,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0

In [16]:
sunshine_grouped.shape

(84, 120)

In [17]:
num_top_venues = 5

for hood in sunshine_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sunshine_grouped[sunshine_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALEXANDRA HEADLAND----
           venue  freq
0           Café  0.29
1  Bowling Alley  0.14
2      BBQ Joint  0.14
3      Surf Spot  0.14
4     Sports Bar  0.14


----AROONA----
                        venue  freq
0  Construction & Landscaping   0.5
1                Home Service   0.5
2                Antique Shop   0.0
3               Movie Theater   0.0
4       Portuguese Restaurant   0.0


----BALD KNOB----
                   venue  freq
0                 Resort  0.25
1  Australian Restaurant  0.25
2                   Café  0.25
3        Thai Restaurant  0.25
4     Photography Studio  0.00


----BALMORAL RIDGE----
                   venue  freq
0                   Park   1.0
1           Antique Shop   0.0
2               Mountain   0.0
3  Portuguese Restaurant   0.0
4                   Pool   0.0


----BATTERY HILL----
          venue  freq
0       RV Park  0.25
1         Beach  0.25
2         Hotel  0.25
3           Bar  0.25
4  Antique Shop  0.00


----BEERWAH----
           v

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sunshine_grouped['Neighborhood']

for ind in np.arange(sunshine_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sunshine_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALEXANDRA HEADLAND,Café,Sports Bar,Surf Spot,BBQ Joint,Bowling Alley,Ice Cream Shop,Dog Run,Donut Shop,Discount Store,French Restaurant
1,AROONA,Construction & Landscaping,Home Service,Fruit & Vegetable Store,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm
2,BALD KNOB,Thai Restaurant,Resort,Australian Restaurant,Café,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
3,BALMORAL RIDGE,Park,Water Park,Indian Restaurant,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm
4,BATTERY HILL,RV Park,Hotel,Beach,Bar,Water Park,Farmers Market,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


In [32]:
# set number of clusters
kclusters = 3

sunshine_grouped_clustering = sunshine_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sunshine_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 1, 0, 2, 2, 0, 2, 0])

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [24]:
sunshine_merged.head()

,Places,Postcode,lat,lon
0,ALEXANDRA HEADLAND,4572,-26.670635,153.108029
1,AROONA,4551,-26.775377,153.113173
2,BALD KNOB,4552,-26.775957,152.892993
3,BALMORAL RIDGE,4552,-26.740350,152.896247
5,BATTERY HILL,4551,-26.777639,153.134820


In [34]:
sunshine_merged = df_merge

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
sunshine_merged = sunshine_merged.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), left_on = sunshine_merged['Places'], right_on='Neighborhood')

sunshine_merged.head() # check the last columns!

,Neighborhood,Places,Postcode,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALEXANDRA HEADLAND,ALEXANDRA HEADLAND,4572,-26.670635,153.108029,2,Café,Sports Bar,Surf Spot,BBQ Joint,Bowling Alley,Ice Cream Shop,Dog Run,Donut Shop,Discount Store,French Restaurant
1,AROONA,AROONA,4551,-26.775377,153.113173,0,Construction & Landscaping,Home Service,Fruit & Vegetable Store,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm
2,BALD KNOB,BALD KNOB,4552,-26.775957,152.892993,2,Thai Restaurant,Resort,Australian Restaurant,Café,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
3,BALMORAL RIDGE,BALMORAL RIDGE,4552,-26.740350,152.896247,1,Park,Water Park,Indian Restaurant,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm
4,BATTERY HILL,BATTERY HILL,4551,-26.777639,153.134820,0,RV Park,Hotel,Beach,Bar,Water Park,Farmers Market,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


In [35]:
# create map
latitude = -26.65
longitude = 153.0667
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sunshine_merged['lat'], sunshine_merged['lon'], sunshine_merged['Neighborhood'], sunshine_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
sunshine_merged[sunshine_merged['Cluster Labels'] == 0].head()

,Neighborhood,Places,Postcode,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALEXANDRA HEADLAND,ALEXANDRA HEADLAND,4572,-26.670635,153.108029,0,Café,Sports Bar,Surf Spot,BBQ Joint,Bowling Alley,Ice Cream Shop,Dog Run,Donut Shop,Discount Store,French Restaurant
2,BALD KNOB,BALD KNOB,4552,-26.775957,152.892993,0,Thai Restaurant,Resort,Australian Restaurant,Café,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
5,BEERWAH,BEERWAH,4519,-26.857043,152.957162,0,Supermarket,Coffee Shop,Bakery,Pub,Burger Joint,Train Station,Asian Restaurant,Grocery Store,Café,Fast Food Restaurant
6,BIRTINYA,BIRTINYA,4575,-26.745513,153.117015,0,Hotel,Café,Mexican Restaurant,Fast Food Restaurant,Thai Restaurant,Sushi Restaurant,Electronics Store,Flower Shop,Fish & Chips Shop,Farmers Market
7,BLACK MOUNTAIN,BLACK MOUNTAIN,4563,-30.308042,151.650342,0,Scenic Lookout,Business Service,Mountain,Monument / Landmark,Water Park,Flower Shop,Dessert Shop,Discount Store,Dog Run,Donut Shop


In [36]:
sunshine_merged[sunshine_merged['Cluster Labels'] == 1].head()

,Neighborhood,Places,Postcode,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,BALMORAL RIDGE,BALMORAL RIDGE,4552,-26.740350,152.896247,1,Park,Water Park,Indian Restaurant,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm
41,HIGHWORTH,HIGHWORTH,4560,-26.620232,152.930627,1,Park,Water Park,Indian Restaurant,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm
65,MUDJIMBA,MUDJIMBA,4564,-26.622195,153.090942,1,Park,Water Park,Indian Restaurant,Department Store,Dessert Shop,Discount Store,Dog Run,Donut Shop,Electronics Store,Farm


In [37]:
sunshine_merged[sunshine_merged['Cluster Labels'] == 2].head()

,Neighborhood,Places,Postcode,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALEXANDRA HEADLAND,ALEXANDRA HEADLAND,4572,-26.670635,153.108029,2,Café,Sports Bar,Surf Spot,BBQ Joint,Bowling Alley,Ice Cream Shop,Dog Run,Donut Shop,Discount Store,French Restaurant
2,BALD KNOB,BALD KNOB,4552,-26.775957,152.892993,2,Thai Restaurant,Resort,Australian Restaurant,Café,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
5,BEERWAH,BEERWAH,4519,-26.857043,152.957162,2,Supermarket,Coffee Shop,Bakery,Pub,Burger Joint,Train Station,Asian Restaurant,Grocery Store,Café,Fast Food Restaurant
6,BIRTINYA,BIRTINYA,4575,-26.745513,153.117015,2,Hotel,Café,Mexican Restaurant,Fast Food Restaurant,Thai Restaurant,Sushi Restaurant,Electronics Store,Flower Shop,Fish & Chips Shop,Farmers Market
10,BLI BLI,BLI BLI,4560,-26.622300,153.041663,2,Grocery Store,Convenience Store,Sandwich Place,Café,Theme Park,Water Park,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
